## Aryo Adi Putro - 2341720084
## TI-3G

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt  # Untuk visualisasi opsional
import requests
from PIL import Image
from io import BytesIO

In [2]:
# Langkah 1: Load dataset langsung dari TensorFlow Datasets
# Split: 80% train, 20% test
(ds_train, ds_test), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    with_info=True,
    as_supervised=True  # Return (image, label)
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.GO5VXY_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


In [3]:
IMG_SIZE = 64
NUM_CLASSES = 1  # Binary classification

def preprocess(image, label):
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])  # Resize
    image = tf.cast(image, tf.float32) / 255.0  # Normalisasi ke [0,1]
    label = tf.cast(label, tf.float32)  # Untuk binary_crossentropy
    return image, label

def augment(image, label):
    image = tf.image.random_flip_left_right(image)  # Horizontal flip
    image = tf.image.random_brightness(image, max_delta=0.2)  # Brightness (mirip shear/zoom)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)  # Contrast (mirip zoom/shear)
    return image, label

In [4]:
ds_train = ds_train.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.map(augment, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.batch(32).prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(32).prefetch(tf.data.AUTOTUNE)

In [5]:
cnn = tf.keras.models.Sequential()

In [6]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [7]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [8]:
cnn.add(tf.keras.layers.Flatten())

In [9]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [10]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [11]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [12]:
history = cnn.fit(
    ds_train,
    validation_data=ds_test,
    epochs=25
)

Epoch 1/25
582/582 ━━━━━━━━━━━━━━━━━━━━ 27s 45ms/step - accuracy: 0.6237 - loss: 0.6422 - val_accuracy: 0.7016 - val_loss: 0.5682
Epoch 2/25
582/582 ━━━━━━━━━━━━━━━━━━━━ 26s 45ms/step - accuracy: 0.7099 - loss: 0.5593 - val_accuracy: 0.7139 - val_loss: 0.5549
Epoch 3/25
582/582 ━━━━━━━━━━━━━━━━━━━━ 26s 45ms/step - accuracy: 0.7389 - loss: 0.5262 - val_accuracy: 0.7491 - val_loss: 0.5093
Epoch 4/25
582/582 ━━━━━━━━━━━━━━━━━━━━ 26s 44ms/step - accuracy: 0.7560 - loss: 0.5012 - val_accuracy: 0.7588 - val_loss: 0.4958
Epoch 5/25
582/582 ━━━━━━━━━━━━━━━━━━━━ 26s 44ms/step - accuracy: 0.7767 - loss: 0.4666 - val_accuracy: 0.7631 - val_loss: 0.4871
Epoch 6/25
582/582 ━━━━━━━━━━━━━━━━━━━━ 27s 47ms/step - accuracy: 0.7871 - loss: 0.4514 - val_accuracy: 0.7693 - val_loss: 0.4908
Epoch 7/25
582/582 ━━━━━━━━━━━━━━━━━━━━ 26s 44ms/step - accuracy: 0.8009 - loss: 0.4283 - val_accuracy: 0.7687 - val_loss: 0.4964
Epoch 8/25
582/582 ━━━━━━━━━━━━━━━━━━━━ 25s 43ms/step - accuracy: 0.8146 - loss: 0.4075 - 

In [25]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

print(f"Hasil prediksi: {prediction}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Hasil prediksi: dog
